In [3]:
# Import packages to use later
import pandas as pd
import nltk
import time
import json

In [4]:
# businesses = pd.read_csv("../input/YelpChallengeWMetros_Clean.csv", encoding='latin-1')
# businesses = pd.read_csv("../input/yelp_business.csv", encoding='latin-1')
# businesses = businesses.rename(columns={'business': 'business_id'})

import pickle
with open('../input/business.pkl', 'rb') as f:
    businesses = pickle.load(f)

In [5]:
d = dict(businesses['city'].value_counts())

for city, count in sorted(d.items(), key=lambda x: x[1], reverse=True):
    if count > 1000:
        print(city, count)

Philadelphia 14569
Tucson 9250
Tampa 9050
Indianapolis 7540
Nashville 6971
New Orleans 6209
Reno 5935
Edmonton 5054
Saint Louis 4827
Santa Barbara 3829
Boise 2937
Clearwater 2221
Saint Petersburg 1663
Metairie 1643
Sparks 1624
Wilmington 1446
Franklin 1414
St. Louis 1255
St. Petersburg 1185
Meridian 1043
Brandon 1033
Largo 1002


In [6]:
businesses.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


In [7]:
# reviews = pd.read_csv("../input/yelp_review.csv", encoding='latin-1')
import pickle
with open('../input/review.pkl', 'rb') as f:
    reviews = pickle.load(f)

In [8]:
reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


In [9]:
# !pip install tqdm

In [10]:
from tqdm import tqdm

# Filter cities containing the majority of reviews and remove outlier entries
major_cities = [city for city, count in sorted(d.items(), key=lambda x: x[1], reverse=True)]

# Create an empty dataframe for each city
city_reviews = {city: pd.DataFrame() for city in major_cities}

# Map city IDs to a set of business IDs in that city
city_businesses = {city: set(businesses[businesses['city'] == city]['business_id'].tolist()) for city in major_cities}

total_review_count = 0

# # Load the review data from the PKL file
# with open('../input/review.pkl', 'rb') as f:
#     reviews = pickle.load(f)

# # Filter and process the reviews
# reviews['datetime'] = pd.to_datetime(reviews['date'])
# reviews = reviews[(reviews['datetime'] >= pd.Timestamp('2010-01-01')) & (reviews['datetime'] < pd.Timestamp('2020-01-01'))]

total_review_count += len(reviews)

# Wrap the major_cities list with tqdm for progress bar
for city in tqdm(major_cities, desc="Processing Cities"):
    relevant_reviews = reviews[reviews.business_id.isin(city_businesses[city])]
    city_reviews[city] = pd.concat([city_reviews[city], relevant_reviews], axis=0)

print(total_review_count)
        
for city in major_cities:
    if len(city_reviews[city]) > 20:
        try:
            city_reviews[city].to_csv(f"../processed/yelp_academic_dataset_reviews_{city}.csv", index=False)
        except Exception as e:
            print(f"Error writing file for {city}: {e}")


Processing Cities: 100%|████████████████████| 1416/1416 [07:01<00:00,  3.36it/s]


6990280
Error writing file for Clearwater/ Countryside: Cannot save file into a non-existent directory: '../processed/yelp_academic_dataset_reviews_Clearwater'
Error writing file for Wayne/Radnor: Cannot save file into a non-existent directory: '../processed/yelp_academic_dataset_reviews_Wayne'


In [11]:
filtered_cities = [city for city in major_cities if len(city_reviews[city]) > 20]
print(filtered_cities)

review_counts = [len(city_reviews[city]) for city in filtered_cities]
print(review_counts)

total_filtered_reviews = sum(review_counts)
print(total_filtered_reviews)

print(total_review_count)

['Philadelphia', 'Tucson', 'Tampa', 'Indianapolis', 'Nashville', 'New Orleans', 'Reno', 'Edmonton', 'Saint Louis', 'Santa Barbara', 'Boise', 'Clearwater', 'Saint Petersburg', 'Metairie', 'Sparks', 'Wilmington', 'Franklin', 'St. Louis', 'St. Petersburg', 'Meridian', 'Brandon', 'Largo', 'Carmel', 'Cherry Hill', 'West Chester', 'Goleta', 'Brentwood', 'Palm Harbor', 'Greenwood', 'New Port Richey', 'Lutz', 'Riverview', 'Kenner', 'Fishers', 'King of Prussia', 'Wesley Chapel', 'Doylestown', 'Pinellas Park', 'Dunedin', 'Hendersonville', 'Bensalem', 'Norristown', 'Exton', 'Marlton', 'Spring Hill', 'Tarpon Springs', 'St Petersburg', 'Springfield', 'Lansdale', 'Ardmore', 'Wayne', 'Media', 'Smyrna', 'Phoenixville', 'Newark', 'Seminole', 'St Louis', 'Gretna', 'Mount Laurel', 'Langhorne', 'Saint Charles', 'Trenton', 'Pottstown', 'Newtown', 'Avon', 'Florissant', 'Willow Grove', 'Havertown', 'Belleville', 'Conshohocken', 'Carpinteria', 'Levittown', 'Sherwood Park', 'Malvern', 'Harvey', 'Eagle', 'Fento